<a href="https://colab.research.google.com/github/thomreid11/DAS7000_PORT1/blob/main/DAS7000_port1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thomas Reid - DAS7000 - PORT1**

Exploratory Data Analysis (EDA) on Manhattan Property Dataset investigating property types and prices within Manhattan, New York

#### **Importing necessary libraries**

In [16]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

#### **Mounting Google Drive to Google Collab and importing dataset**

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
drive_path = '/content/drive/MyDrive/Manhattan property dataset csv.csv'

In [19]:
# Converting csv to df
df = pd.read_csv(drive_path)

In [20]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,376,43,NaN,S1,743 EAST 6 STREET,NaN,...,1.0,1.0,2.0,"2,090","3,680",1940.0,1,S1,0,01/23/2025
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,372,19,NaN,C7,"292 EAST 3 STREET, 4B",NaN,...,9.0,1.0,10.0,"2,401","6,920",1920.0,2,C7,0,02/07/2025
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,396,26,NaN,C4,638 EAST 14 STREET,NaN,...,20.0,0.0,20.0,"2,779","10,740",1920.0,2,C4,0,05/01/2025
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,396,27,NaN,C4,640 EAST 14 STREET,NaN,...,20.0,0.0,20.0,"2,831","10,900",1920.0,2,C4,0,05/01/2025
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,400,11,NaN,C4,"510 EAST 5TH STREET, 9",NaN,...,18.0,0.0,18.0,"2,404","6,875",1900.0,2,C4,"280,000",04/29/2025


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18491 entries, 0 to 18490
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         18491 non-null  int64  
 1   NEIGHBORHOOD                    18491 non-null  object 
 2   BUILDING CLASS CATEGORY         18491 non-null  object 
 3   TAX CLASS AT PRESENT            18491 non-null  object 
 4   BLOCK                           18491 non-null  int64  
 5   LOT                             18491 non-null  int64  
 6   EASEMENT                        0 non-null      float64
 7   BUILDING CLASS AT PRESENT       18491 non-null  object 
 8   ADDRESS                         18491 non-null  object 
 9   APARTMENT NUMBER                8634 non-null   object 
 10  ZIP CODE                        18491 non-null  int64  
 11  RESIDENTIAL UNITS               9366 non-null   float64
 12  COMMERCIAL UNITS                

#### **Data Preprocessing**

In [22]:
# Making columns neat and uniform
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.title()

In [23]:
# Correcting the data types
# Changing strings to floats and removing the commas
df['Land_Square_Feet'] = df['Land_Square_Feet'].str.replace(',', '', regex=False).astype(float)
df['Gross_Square_Feet'] = df['Gross_Square_Feet'].str.replace(',', '', regex=False).astype(float)
df['Sale_Price'] = df['Sale_Price'].str.replace(',', '', regex=False).astype(float)

# Changing the remaining data types
df['Sale_Date'] = pd.to_datetime(df['Sale_Date'])
df["Tax_Class_At_Time_Of_Sale"] = df["Tax_Class_At_Time_Of_Sale"].astype("string")
df["Tax_Class_At_Present"] = df["Tax_Class_At_Present"].astype("string")


# Changing 3 columns in one go for efficiency
unit_cols = ["Residential_Units", "Commercial_Units", "Total_Units"]
for col in unit_cols:
    df[col] = df[col].astype("Int64")


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18491 entries, 0 to 18490
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Borough                         18491 non-null  int64         
 1   Neighborhood                    18491 non-null  object        
 2   Building_Class_Category         18491 non-null  object        
 3   Tax_Class_At_Present            18491 non-null  string        
 4   Block                           18491 non-null  int64         
 5   Lot                             18491 non-null  int64         
 6   Easement                        0 non-null      float64       
 7   Building_Class_At_Present       18491 non-null  object        
 8   Address                         18491 non-null  object        
 9   Apartment_Number                8634 non-null   object        
 10  Zip_Code                        18491 non-null  int64         
 11  Re

In [25]:
# Number of Rows
df.shape[0]

18491

In [26]:
# Number of Columns
df.shape[1]

21

We can see that there are 20 columns and 18491. The list shows that not all columns have the full amount of data. The amount of entries missing per column will be worked out and then if it is over 50%, the column will be dropped. If not then it will be interpolated.

In [27]:
# Finding out the percentage of missing values per column
def missing_values_percentage_table(df):

# Finding the number of missing values per column
    missing_values = df.isnull().sum()

# Finding the percentage of missing values by dividing it by the number of data entries there should be and multiplying by 100
    missing_value_perc =  (100 * missing_values / len(df))

# Concatenating the missing values and missing values %  along the top of the table
    missing_value_table = pd.concat([missing_values, missing_value_perc], axis=1)
    missing_value_table = missing_value_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})

# Sort the table by percentage of missing descending
    missing_values_table = missing_value_table.sort_values('% of Total Values', ascending=False)
    return missing_values_table

missing_percentage_df = missing_values_percentage_table(df)


display(missing_percentage_df.style.background_gradient(cmap='Blues'))

,Missing Values,% of Total Values
Easement,18491,100.000000
Land_Square_Feet,16963,91.736520
Gross_Square_Feet,16963,91.736520
Commercial_Units,16171,87.453356
Apartment_Number,9857,53.307014
Residential_Units,9125,49.348332
Total_Units,8333,45.065167
Year_Built,1936,10.469958
Block,0,0.000000
Borough,0,0.000000


In [29]:
# Omitting Easement, Land_Square_Feet, Gross_Square_Feet, Commerical_Units, Apartment_number, Residential_Units

cols_to_drop = [
    "Easement",
    "Land_Square_Feet",
    "Gross_Square_Feet",
    "Commercial_Units",
    "Apartment_Number",
    "Residential_Units"
]

df = df.drop(columns=cols_to_drop)

Total Units - Interpolated through using the median Total Units of the neighbourhood

Year Built - Interpolated through using the modal Year Built of building category

In [34]:
# Interpolating Total Units
df["Total_Units"] = df.groupby("Neighborhood")["Total_Units"]\
                      .transform(lambda x: x.fillna(x.median()))

In [36]:
# Interpolating Year Built
df["Year_Built"] = df.groupby("Building_Class_Category")["Year_Built"]\
                     .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.median()))


In [ ]:
# Making new csv with updated data entries